# AML Project - Anomaly Segmentation

## Notebook setup

In [ ]:
%cd /content
!rm -r AnomalySegmentation_CourseProjectBaseCode
!git clone https://github.com/AML-project-AnomalySegmentation/AnomalySegmentation_CourseProjectBaseCode.git


In [ ]:
!pip3 install visdom
!pip3 install ood_metrics
!pip3 install cityscapesScripts


## Steps

In [ ]:
%cd /content/AnomalySegmentation_CourseProjectBaseCode/eval
!python converter.py

### Step 2A - Compute AuPRC & FPR95TPR

In [ ]:
DATASETS = ["RoadAnomaly21", "RoadObsticle21", "FS_LostFound_full", "fs_static", "RoadAnomaly"]
DISCRIMINANTS = ["msp", "maxlogit", "maxentropy"]
for d in (DATASETS):
  path = f"'Validation_Dataset/{d}/images/*.*'"
  for disc in (DISCRIMINANTS):
    print(f"Dataset: {d} Discriminant: {disc}")
    !python evalAnomaly.py  --input {path} --discriminant {disc}

In [ ]:
DISCRIMINANTS = ["msp", "maxlogit", "maxentropy"]

for d in DISCRIMINANTS:
   !python eval_iou.py --discriminant {d}

### Step 2B - Compute AuPRC & FPR95TPR with temperatures

In [ ]:
DATASETS = ["RoadAnomaly21", "RoadObsticle21", "FS_LostFound_full", "fs_static", "RoadAnomaly"]
TEMPERATURES = [1,0.5,0.75,1.1]
for d in (DATASETS):
  path = f"'Validation_Dataset/{d}/images/*.*'"
  for t in (TEMPERATURES):
    print(f"Dataset: {d} Temeperature: {t}")
    !python evalAnomaly.py  --input {path} --temperature {t}


In [ ]:
!python optimalTemperature.py

In [ ]:
DATASETS = ["RoadAnomaly21", "RoadObsticle21", "FS_LostFound_full", "fs_static", "RoadAnomaly"]
OPTIMAL_TEMPERATURE = 2.13
for d in (DATASETS):
    path = f"'Validation_Dataset/{d}/images/*.*'"
    print(f"Dataset: {d} Temeperature: {t}")
    !python evalAnomaly.py  --input {path} --temperature {OPTIMAL_TEMPERATURE}

In [ ]:
TEMPERATURE = 2.13
for t in TEMPERATURE:
   !python eval_iou.py --temperature {t} --discriminant "msp"

## Step 3 - Train models with void class

In [ ]:
%cd /content/AnomalySegmentation_CourseProjectBaseCode/train

### Train & Compute AuPRC & FPR95TPR Bisenet

In [ ]:
!python main_training_void.py --savedir "/trainingdata" --resume --epochs-save 1 --num-epochs 3 --decoder --model 'bisenetv1'

In [ ]:
!python void_eval_iou.py --model "BiseNet"

In [ ]:
DATASETS = ["RoadAnomaly21", "RoadObsticle21", "FS_LostFound_full", "fs_static", "RoadAnomaly"]
for d in (DATASETS):
    path = f"'Validation_Dataset/{d}/images/*.*'"
    print(f"Dataset: {d}")
    !python VoidEvalAnomaly.py  --input {path} --model "BiseNet"

### Train & Compute AuPRC & FPR95TPR ENet

In [ ]:
!python main_training_void.py --savedir "/trainingdata" --resume --epochs-save 1 --num-epochs 3 --decoder --model 'enet'

In [ ]:
!python void_eval_iou.py --model "ENet"

In [ ]:
DATASETS = ["RoadAnomaly21", "RoadObsticle21", "FS_LostFound_full", "fs_static", "RoadAnomaly"]
for d in (DATASETS):
    path = f"'Validation_Dataset/{d}/images/*.*'"
    print(f"Dataset: {d}")
    !python VoidEvalAnomaly.py  --input {path} --model "ENet"

### Train & Compute AuPRC & FPR95TPR ErfNet

In [ ]:
!python main_training_void.py --savedir "/trainingdata" --resume --epochs-save 1 --num-epochs 3 --decoder --model 'erfnet'

In [ ]:
!python void_eval_iou.py --model "ErfNet"

In [ ]:
DATASETS = ["RoadAnomaly21", "RoadObsticle21", "FS_LostFound_full", "fs_static", "RoadAnomaly"]
for d in (DATASETS):
    path = f"'Validation_Dataset/{d}/images/*.*'"
    print(f"Dataset: {d}")
    !python VoidEvalAnomaly.py  --input {path} --model "ErfNet"

In [ ]:
DATASETS = ["RoadAnomaly21", "RoadObsticle21", "FS_LostFound_full", "fs_static", "RoadAnomaly"]
for d in (DATASETS):
    path = f"'Validation_Dataset/{d}/images/*.*'"
    print(f"Dataset: {d}")
    !python VoidEvalAnomaly.py  --input {path} --model "ErfNet"


## Step 4 - Train ErfNet with addtional losses

## Train ErfNet


#### Logit Normalization

In [ ]:
%cd /content/AnomalySegmentation_CourseProjectBaseCode/train
!python main.py --savedir "/trainingdata_LogitNorm" --resume --epochs-save 1 --num-epochs 3 --decoder --customloss 'LogitNorm' --onlycustom

##### Compute AuPRC & FPR95TPR

In [ ]:
%cd /content/AnomalySegmentation_CourseProjectBaseCode/eval
DATASETS = ["RoadAnomaly21", "RoadObsticle21", "FS_LostFound_full", "fs_static", "RoadAnomaly"]
for d in (DATASETS):
  path = f"'Validation_Dataset/{d}/images/*.*'"
  print(f"Dataset: {d}")
  !python evalAnomaly.py  --input {path} --discriminant "msp" --loadWeights "model_best_only_logit.pth"

##### Compute mIoU

In [ ]:
%cd /content/AnomalySegmentation_CourseProjectBaseCode/eval
!python eval_iou.py --discriminant "msp" --loadWeights "model_best_only_logit.pth"

#### Logit Normalization + Cross Entropy Loss

In [ ]:
%cd /content/AnomalySegmentation_CourseProjectBaseCode/train
!python main.py --savedir "/trainingdata_LogitNormPlusCross" --resume --epochs-save 1 --num-epochs 3 --decoder --customloss 'LogitNorm'

##### Compute AuPRC & FPR95TPR

In [ ]:
%cd /content/AnomalySegmentation_CourseProjectBaseCode/eval
DATASETS = ["RoadAnomaly21", "RoadObsticle21", "FS_LostFound_full", "fs_static", "RoadAnomaly"]
for d in (DATASETS):
  path = f"'Validation_Dataset/{d}/images/*.*'"
  print(f"Dataset: {d}")
  !python evalAnomaly.py  --input {path} --discriminant "msp" --loadWeights "model_best_crs_logit.pth"

##### Compute mIoU

In [ ]:
%cd /content/AnomalySegmentation_CourseProjectBaseCode/eval
!python eval_iou.py --discriminant "msp" --loadWeights "model_best_crs_logit.pth"

#### Logit Normalization + Focal Loss

In [ ]:
%cd /content/AnomalySegmentation_CourseProjectBaseCode/train
!python main.py --savedir "/trainingdata_LogitNormPlusFocal" --resume --epochs-save 1 --num-epochs 3 --decoder --customloss 'LogitNorm' --focal

##### Compute AuPRC & FPR95TPR

In [ ]:
%cd /content/AnomalySegmentation_CourseProjectBaseCode/eval
DATASETS = ["RoadAnomaly21", "RoadObsticle21", "FS_LostFound_full", "fs_static", "RoadAnomaly"]
for d in (DATASETS):
  path = f"'Validation_Dataset/{d}/images/*.*'"
  print(f"Dataset: {d}")
  !python evalAnomaly.py  --input {path} --discriminant "msp" --loadWeights "model_best_focal_logit.pth"

##### Compute mIoU

In [ ]:
%cd /content/AnomalySegmentation_CourseProjectBaseCode/eval
!python eval_iou.py --discriminant "msp" --loadWeights "model_best_focal_logit.pth"

#### Enhanced Isotropy Maximization Loss

In [ ]:
%cd /content/AnomalySegmentation_CourseProjectBaseCode/train
!python main.py --savedir "/trainingdata_IsoMax" --resume --epochs-save 1 --num-epochs 3 --decoder --customloss 'IsoMax' --onlycustom

##### Compute AuPRC & FPR95TPR

In [ ]:
%cd /content/AnomalySegmentation_CourseProjectBaseCode/eval
DATASETS = ["RoadAnomaly21", "RoadObsticle21", "FS_LostFound_full", "fs_static", "RoadAnomaly"]
for d in (DATASETS):
  path = f"'Validation_Dataset/{d}/images/*.*'"
  print(f"Dataset: {d}")
  !python evalAnomaly.py  --input {path} --discriminant "msp" --loadWeights "model_best_only_isomax.pth"

##### Compute mIoU

In [ ]:
%cd /content/AnomalySegmentation_CourseProjectBaseCode/eval
!python eval_iou.py --discriminant "msp" --loadWeights "model_best_only_isomax.pth"

#### Enhanced Isotropy Maximization Loss + Cross Entropy Loss

In [ ]:
%cd /content/AnomalySegmentation_CourseProjectBaseCode/train
!python main.py --savedir "/trainingdata_IsoMaxPlusCross" --resume --epochs-save 1 --num-epochs 3 --decoder --customloss 'IsoMax' 

##### Compute AuPRC & FPR95TPR

In [ ]:
%cd /content/AnomalySegmentation_CourseProjectBaseCode/eval
DATASETS = ["RoadAnomaly21", "RoadObsticle21", "FS_LostFound_full", "fs_static", "RoadAnomaly"]
for d in (DATASETS):
  path = f"'Validation_Dataset/{d}/images/*.*'"
  print(f"Dataset: {d}")
  !python evalAnomaly.py  --input {path} --discriminant "msp" --loadWeights "model_best_crs_isomax.pth"

##### Compute mIoU

In [ ]:
%cd /content/AnomalySegmentation_CourseProjectBaseCode/eval
!python eval_iou.py --discriminant "msp" --loadWeights "model_best_crs_isomax.pth"


#### Enhanced Isotropy Maximization Loss + Focal Loss

In [ ]:
%cd /content/AnomalySegmentation_CourseProjectBaseCode/train

!python main.py --savedir "/trainingdata_IsoMaxPlusFocal" --resume --epochs-save 1 --num-epochs 3 --decoder --customloss 'IsoMax' --focal

##### Compute AuPRC & FPR95TPR

In [ ]:
%cd /content/AnomalySegmentation_CourseProjectBaseCode/eval
!python eval_iou.py --discriminant "msp" --loadWeights "model_best_focal_isomax.pth"

##### Compute mIoU

In [ ]:
%cd /content/AnomalySegmentation_CourseProjectBaseCode/eval
DATASETS = ["RoadAnomaly21", "RoadObsticle21", "FS_LostFound_full", "fs_static", "RoadAnomaly"]
for d in (DATASETS):
  path = f"'Validation_Dataset/{d}/images/*.*'"
  print(f"Dataset: {d}")
  !python evalAnomaly.py  --input {path} --discriminant "msp" --loadWeights "model_best_focal_isomax.pth"